In [5]:
import torch
import os
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_curve

In [6]:
from shutil import copy2

In [7]:
import matplotlib.pyplot as plt

In [8]:
races = ['African','Asian','Caucasian','Indian']
genders = ['Man','Woman']
labels = []
dist = []
for race in races:
    for gender in genders:
        with open(f"./sims/{race}_{gender}_sims.csv",'r') as f:
            lines = f.readlines()
        for line in lines:
            arr = line.strip().split(',')
            labels.append(int(arr[3]))
            dist.append(float(arr[2]))

In [9]:
fpr , tpr ,thresholds = roc_curve(labels,dist)
idx = np.argmax(tpr-fpr)
threshold = thresholds[idx]
print(f"Threshold: {threshold}")

Threshold: 0.30252805352211


In [10]:
races = ['African','Asian','Caucasian','Indian']
genders = ['Man','Woman']
accuracies = {}
for race in races:
    for gender in genders:
        accuracies[f"{race}_{gender}"] = {'TP':0,'TN':0,'FP':0,'FN':0}
        with open(f"./sims/{race}_{gender}_sims.csv",'r') as f:
            lines = f.readlines()
        dist = []
        labels = []
        for line in lines:
            arr = line.strip().split(',')
            dist.append(float(arr[2]))
            labels.append(int(arr[3]))
            if float(arr[2]) < threshold:
                with open(f"./results/{race}_{gender}_results.csv",'a') as f:
                    f.write(f"{arr[0]},{arr[1]},0,{arr[3]}\n")
                if int(arr[3]) == 1:
                    accuracies[f"{race}_{gender}"]['TN'] += 1
                else:
                    accuracies[f"{race}_{gender}"]['FN'] += 1
            else:
                with open(f"./results/{race}_{gender}_results.csv",'a') as f:
                    f.write(f"{arr[0]},{arr[1]},1,{arr[3]}\n")
                if int(arr[3]) == 1:
                    accuracies[f"{race}_{gender}"]['TP'] += 1
                else:
                    accuracies[f"{race}_{gender}"]['FP'] += 1
        fpr, tpr, thresholds = roc_curve(labels,dist)
        fpr = np.array(fpr)
        tpr = np.array(tpr)
        with open(f"./roc/{race}_{gender}_roc.npy",'wb') as f:
            np.save(f,fpr)
            np.save(f,tpr)

In [11]:
print(accuracies)

{'African_Man': {'TP': 2906, 'TN': 33, 'FP': 32, 'FN': 2908}, 'African_Woman': {'TP': 58, 'TN': 3, 'FP': 0, 'FN': 60}, 'Asian_Man': {'TP': 2084, 'TN': 44, 'FP': 14, 'FN': 2117}, 'Asian_Woman': {'TP': 830, 'TN': 42, 'FP': 8, 'FN': 861}, 'Caucasian_Man': {'TP': 2205, 'TN': 48, 'FP': 0, 'FN': 2252}, 'Caucasian_Woman': {'TP': 723, 'TN': 24, 'FP': 1, 'FN': 747}, 'Indian_Man': {'TP': 2266, 'TN': 49, 'FP': 17, 'FN': 2300}, 'Indian_Woman': {'TP': 665, 'TN': 20, 'FP': 10, 'FN': 672}}


In [12]:
gar = {}
far = {}
scores = {}
for race in races:
    for gender in genders:
        tp = accuracies[f"{race}_{gender}"]['TP']
        tn = accuracies[f"{race}_{gender}"]['TN']
        fp = accuracies[f"{race}_{gender}"]['FP']
        fn = accuracies[f"{race}_{gender}"]['FN']
        gar[f"{race}_{gender}"] = tp/(tn+tp)
        far[f"{race}_{gender}"] = fp/(fn+fp)
        scores[f"{race}_{gender}"] = 100*(fn+tp)/(fn+fp+tn+tp)

In [13]:
gar

{'African_Man': 0.988771691051378,
 'African_Woman': 0.9508196721311475,
 'Asian_Man': 0.9793233082706767,
 'Asian_Woman': 0.9518348623853211,
 'Caucasian_Man': 0.9786950732356857,
 'Caucasian_Woman': 0.9678714859437751,
 'Indian_Man': 0.9788336933045356,
 'Indian_Woman': 0.9708029197080292}

In [14]:
far

{'African_Man': 0.010884353741496598,
 'African_Woman': 0.0,
 'Asian_Man': 0.00656968559361802,
 'Asian_Woman': 0.009205983889528193,
 'Caucasian_Man': 0.0,
 'Caucasian_Woman': 0.001336898395721925,
 'Indian_Man': 0.0073370738023306,
 'Indian_Woman': 0.01466275659824047}

In [15]:
scores

{'African_Man': 98.89436979078074,
 'African_Woman': 97.52066115702479,
 'Asian_Man': 98.63817797605071,
 'Asian_Woman': 97.12808730614589,
 'Caucasian_Man': 98.93451720310766,
 'Caucasian_Woman': 98.32775919732441,
 'Indian_Man': 98.57512953367876,
 'Indian_Woman': 97.8054133138259}